In [ ]:
from langchain.llms import OpenAI
from langchain.agents import Tool
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import initialize_agent, AgentType

In [ ]:
import os

# Initialize variables to store the API keys
google_api_key = None
google_cse_id = None

# Read the API keys from the file
with open('APIkeys.txt', 'r') as file:
    for line in file:
        if line.startswith('google_api:'):
            google_api_key = line.strip().split(': ')[1]
        elif line.startswith('google_cse:'):
            google_cse_id = line.strip().split(': ')[1]

# Check if the keys were found and set them as environment variables
if google_api_key and google_cse_id:
    os.environ["GOOGLE_API_KEY"] = google_api_key
    os.environ["GOOGLE_CSE_ID"] = google_cse_id
    print("Environment variables set successfully.")
else:
    print("Failed to find required API keys in the file.")

In [ ]:
llm = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)

In [ ]:
search = GoogleSearchAPIWrapper()

prompt = PromptTemplate(
    input_variables=["query"],
    template="Write a summary of the following text: {query}"
)

summarize_chain = LLMChain(llm=llm, prompt=prompt)

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for finding information about recent events"
    ),
    Tool(
       name='Summarizer',
       func=summarize_chain.run,
       description='useful for summarizing texts'
    )
]

In [ ]:
agent = initialize_agent(tools, 
                         llm, 
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
                         verbose=True,
                         max_iterations=6)

In [ ]:
response = agent("What's the latest news about the Mars rover?")
print(response['output'])